### Final Evaluation:

I will try the last 4 iterations of the recommendation engine again and use more games to choose one. I will use only the last 4 models as BERT is clearly inferior.

In [2]:
import pandas as pd
import gc
from fractions import Fraction

In [3]:
sg_df_clean = pd.read_csv("sg_df_clean.csv")

In [4]:
gc.collect()

0

In [5]:
sg_df_clean["rating_ratio"] = sg_df_clean["rating"]/100
print(sg_df_clean["rating_ratio"])

0        0.352941
1        0.913793
2        1.000000
3        0.862069
4        0.639706
           ...   
72366    0.808989
72367    0.681199
72368    1.000000
72369    1.000000
72370    0.666667
Name: rating_ratio, Length: 72371, dtype: float64


In [6]:
c = sg_df_clean["rating_ratio"].mean()
m= sg_df_clean["user_reviews"].median()

In [7]:
def weighted_game_score(x, c=c, m=m):
    r = x["rating_ratio"] #I am taking the rating_ratio for each game (row) and storing it as variable r
    n = x["user_reviews"]#I am taking the user_reviews for each game (row) and storing it as variable n
    return ((n*r) / (n+m) + (m*c) / (n+m))

In [8]:
sg_df_clean["game_score"] = sg_df_clean.apply(weighted_game_score, axis=1)

In [9]:
sg_df_clean[["name","game_score"]].sample(20)

,name,game_score
49496,The Blueness of a Wound,0.749422
62932,The Symbiant Re:Union,0.919218
33779,Fantastic Contraption,0.777542
52431,Darkened Glory,0.794876
57784,Merchant of the Six Kingdoms,0.854428
50157,Gun Witch,0.901207
22117,Time Of Special Team(T.O.S.T),0.703967
62049,Choice Clash: What Would You Rather?,0.737490
39269,Zombie Panic! Source,0.847797
36774,The Wilting Amaranth,0.827636


In [10]:
def recommend_games(game, sim_matrix, df = sg_df_clean):
    # 1. Find the game's index in the dataframe to use it in the similarity matrix
    try: 
        index = df[df['name'] == game].index[0]
    except IndexError:
        return "The game you typed does not exist in the database. Please make sure the spelling exactly matches the game on steam" #to give an error if the spelling is incorrect
    
    # 2. We create a temp dataframe to modify without changing the original
    temp_df = df.copy()
    
    # 3. We create a new column with the list of cosine similarities for the specified game
    temp_df['similarity'] = sim_matrix[index]
    
    # 4. FILTER (Get top 20 matches)
    # Sort by similarity (Descending)
    # iloc[1:21] grabs the top 20, skipping the game itself (which is the first one since it's the most similar)
    top_similar = temp_df.sort_values('similarity', ascending=False).iloc[1:21]
    
    # 4. RANK (Pick top 5 best quality)
    # Sort the 20 candidates by 'game_score' and pick the top 5
    top_picks = top_similar.sort_values('game_score', ascending=False).head(5)
    
    # 5. Return only the relevant columns
    cols = ['name', 'similarity', 'game_score']
    return top_picks[cols]



I will create a new function to check recommendations, this time it will have 15 games instead of 5

In [11]:
def rec_rating(x):
    # Define the list of test games
    test_games = [
        "The Elder Scrolls V: Skyrim Special Edition", "DOOM Eternal", 
        "Hollow Knight", "Hades", "ELDEN RING", "Cyberpunk 2077", "SOMA", 
        "ULTRAKILL", "Sekiro™: Shadows Die Twice - GOTY Edition", "Dota 2", "DUSK", 
        "Borderlands Game of the Year", "CHRONO TRIGGER®", 
        "Divinity: Original Sin 2 - Definitive Edition", "Destiny 2"
    ]
    
    #Loop through and print
    for game in test_games:
        print(f"{game}:")
        # call the function and print the result
        print(recommend_games(game=game, sim_matrix=x))
        
        # This is for formatting
        print("\n" + "="*40 + "\n")


#### 1: ModernBertEmbed Evaluation:

In [13]:
Full_cosine_matrix = pd.read_pickle("Full_cosine_matrix_modernbertembed.pkl")

In [14]:
rec_rating(Full_cosine_matrix)

The Elder Scrolls V: Skyrim Special Edition:
                                                    name  similarity  \
16413  The Elder Scrolls IV: Oblivion® Game of the Ye...    0.532999   
51296  The Elder Scrolls IV: Oblivion® Game of the Ye...    0.537822   
5785   The Elder Scrolls III: Morrowind® Game of the ...    0.493882   
9109                         The Elder Scrolls V: Skyrim    0.616182   
63572           Horizon Forbidden West™ Complete Edition    0.436708   

       game_score  
16413    0.955539  
51296    0.955496  
5785     0.953199  
9109     0.948258  
63572    0.938925  


DOOM Eternal:
                       name  similarity  game_score
3401          Ultimate Doom    0.563734    0.964302
33218         Devil Daggers    0.491746    0.955320
7623                   DOOM    0.551795    0.952628
17204  Warstride Challenges    0.531242    0.917679
21004         Dread Templar    0.517306    0.908092


Hollow Knight:
                                         name  similarity

In [15]:
modernbert_score = (5 + 5 + 4 + 5 + 3 + 3 + 3 + 4 + 4 + 3 + 4 + 4 + 5 + 5 + 3) / (15*5)
modernbert_score

0.8

In [16]:
del Full_cosine_matrix
gc.collect()

0

#### 2: EmbeddingGemma Evaluation:

In [18]:
Full_cosine_matrix = pd.read_pickle("Full_cosine_matrix_EmbeddingGemma.pkl")

In [19]:
rec_rating(Full_cosine_matrix)

The Elder Scrolls V: Skyrim Special Edition:
                                                    name  similarity  \
16413  The Elder Scrolls IV: Oblivion® Game of the Ye...    0.507183   
51296  The Elder Scrolls IV: Oblivion® Game of the Ye...    0.499036   
5785   The Elder Scrolls III: Morrowind® Game of the ...    0.439858   
9109                         The Elder Scrolls V: Skyrim    0.638590   
3765                                Sid Meier's Pirates!    0.390450   

       game_score  
16413    0.955539  
51296    0.955496  
5785     0.953199  
9109     0.948258  
3765     0.941963  


DOOM Eternal:
                name  similarity  game_score
3401   Ultimate Doom    0.530816    0.964302
33218  Devil Daggers    0.464657    0.955320
7623            DOOM    0.534766    0.952628
23005        DOOM II    0.470500    0.948779
53559    HYPER DEMON    0.466565    0.941629


Hollow Knight:
                                name  similarity  game_score
13307  Shovel Knight: Treasure Trove  

In [20]:
EmbeddingGemma_score = (4 + 5 + 4 + 5 + 4 + 5 + 4 + 5 + 3 + 2 + 4 + 5 + 5 + 5 + 4) / (15*5)
EmbeddingGemma_score

0.8533333333333334

In [21]:
del Full_cosine_matrix
gc.collect()

0

#### 3: Qwen3-embedding-0.6B V1 Evaluation:

In [23]:
Full_cosine_matrix = pd.read_pickle("Full_cosine_matrix_Qwen-3-6B.pkl")

In [24]:
rec_rating(Full_cosine_matrix)

The Elder Scrolls V: Skyrim Special Edition:
                                                    name  similarity  \
16413  The Elder Scrolls IV: Oblivion® Game of the Ye...    0.569550   
51296  The Elder Scrolls IV: Oblivion® Game of the Ye...    0.554443   
5785   The Elder Scrolls III: Morrowind® Game of the ...    0.544408   
9109                         The Elder Scrolls V: Skyrim    0.577546   
63572           Horizon Forbidden West™ Complete Edition    0.559197   

       game_score  
16413    0.955539  
51296    0.955496  
5785     0.953199  
9109     0.948258  
63572    0.938925  


DOOM Eternal:
                     name  similarity  game_score
12387                DUSK    0.574517    0.978824
48612      Shadow Warrior    0.555321    0.925725
60722       Missiles Away    0.578699    0.895053
18318  Serious Sam 3: BFE    0.562023    0.884007
11386    Shadow Warrior 2    0.558709    0.876779


Hollow Knight:
                              name  similarity  game_score
58738     

In [25]:
Qwen3_V1_score = (5 + 5 + 4 + 5 + 3 + 3 + 4 + 3 + 5 + 2 + 4 + 5 + 3 + 3 + 4) / (15*5)
Qwen3_V1_score

0.7733333333333333

In [26]:
del Full_cosine_matrix
gc.collect()

0

#### 4: Qwen3-embedding-0.6B V2 Evaluation:

In [28]:
Full_cosine_matrix = pd.read_pickle("Full_cosine_matrix_Qwen-3-6B-V2.pkl")

In [29]:
rec_rating(Full_cosine_matrix)

The Elder Scrolls V: Skyrim Special Edition:
                                                    name  similarity  \
5785   The Elder Scrolls III: Morrowind® Game of the ...    0.600299   
9109                         The Elder Scrolls V: Skyrim    0.693637   
3765                                Sid Meier's Pirates!    0.619610   
63572           Horizon Forbidden West™ Complete Edition    0.596526   
32158                   Middle-earth™: Shadow of Mordor™    0.593767   

       game_score  
5785     0.953199  
9109     0.948258  
3765     0.941963  
63572    0.938925  
32158    0.922377  


DOOM Eternal:
                name  similarity  game_score
3401   Ultimate Doom    0.609788    0.964302
57770       Trepang2    0.599833    0.954439
7623            DOOM    0.603032    0.952628
15523        DOOM 64    0.594878    0.938437
29452      Shrine II    0.589017    0.930044


Hollow Knight:
                                  name  similarity  game_score
31391      Silver Axe - The Honest E

In [30]:
Qwen3_V2_score = (3 + 5 + 5 + 5 + 4 + 3 + 2 + 3 + 3 + 4 + 4 + 5 + 2 + 4 + 3) / (15*5)
Qwen3_V2_score

0.7333333333333333

In [31]:
del Full_cosine_matrix
gc.collect()

0

#### 5: Qwen3-embedding-0.6B V3 Evaluation:

In [33]:
Full_cosine_matrix = pd.read_pickle("Full_cosine_matrix_Qwen-3-6B-V3.pkl")

In [34]:
rec_rating(Full_cosine_matrix)

The Elder Scrolls V: Skyrim Special Edition:
                                                    name  similarity  \
5785   The Elder Scrolls III: Morrowind® Game of the ...    0.411369   
9109                         The Elder Scrolls V: Skyrim    0.514083   
4360                                         RuneScape ®    0.410125   
15646                       Mount & Blade II: Bannerlord    0.407139   
38656                                             ADONIS    0.409396   

       game_score  
5785     0.953199  
9109     0.948258  
4360     0.879675  
15646    0.876779  
38656    0.848541  


DOOM Eternal:
                                                    name  similarity  \
3401                                       Ultimate Doom    0.463865   
7623                                                DOOM    0.423732   
53559                                        HYPER DEMON    0.422673   
15523                                            DOOM 64    0.421294   
42470  Demon Slayer -Kimet

In [35]:
Qwen3_V3_score = (4 + 4 + 4 + 5 + 2 + 4 + 4 + 5 + 4 + 5 + 4 + 3 + 5 + 3) / (15*5)
Qwen3_V3_score

0.7466666666666667

### Conclusion:

| Model | Score |
| :--- | :--- |
| EmbeddingGemma | 0.853 |
| ModernBertEmbed | 0.8 |
| Qwen3 V1 | 0.773 |
| Qwen3 V3 | 0.7467 |
| Qwen3 V2 | 0.733 |

EmbeddingGemma managed to recommend much better games with modernbert following right behind it. Qwen3 models however fell short with a lot of recommendations being completely differentated. 

Possible reasons include:

1. The length of the documents: The "about the game" section is not a lengthy document that requires a lot of context, which is what qwen3-embed 0.6B excels at.
   
2. EmbeddingGemma is bidirectional while qwen3-embed reads text from left to right only. It could miss context that EmbeddingGemma would capture. MoidernBertEmbed is also bidirectional which could explain the better results than qwen3-embed.
   
3. The number of tokens, due to ram limitations, qwen3-embed cannot use 32k tokens. I used 512 tokens for all models.

Overall, the EmbeddingGemma recommendations engine is very viable as a tool for steam itself or third party sites. It could help users looking for hidden gems especially if their interested in niche genres. This would help increase the revenue for developers who struggle with visibility, help steam diversify the sources of revenue and help users find games that match their interest.

##### Limitations:

1. The project relies a lot on user reviews which can be unreliable. For example, a very well regarded game critically could be given a low user score due to sociopolitics or bugs. The opposite could also be true as niche games may have a high percentage of positive reviews which may not reflect the quality of the game.

2. "about the game" is an arbitrary way to compare games since it is a free form way for the developers to describe their game. Very similar games may not have the same description.

3. The model uses texts only to find similar games, when in reality video games are more complex and video data would probably reflect the game better.

4. Tags are also user defined and may not be the best way to "frame" a game.

5. Lack of user data, which could be helpful for detecting patterns for similar tastes.

6. The data is not updated, it is a bit more than a year old.

7. The recommendations engine is "static" as adding more data would require repeating the whole process.

8. The scoring system for recommendations is arbitrary as it relies on my subjective opinion of what a good recommendation is. Ideally, it would be tested by users on a larger scale.ated. 

### References

__Dataset__

1. https://www.kaggle.com/datasets/fronkongames/steam-games-dataset
   
__Models__

1. https://huggingface.co/nomic-ai/modernbert-embed-base

2. https://huggingface.co/google/embeddinggemma-300m

3. https://huggingface.co/Qwen/Qwen3-Embedding-0.6B

4. https://huggingface.co/google-bert/bert-base-uncased

__Libraries__

1. Pandas
2. Numpy
3. ast
4. gc
5. SentenceTransformers
6. Matplotlib
7. Seaborn
8. Transformers
9. Langid
10. sys
11. Pytorch
12. scikit-learn
13. wordcloud
14. huggingface_hub